In [1]:
# 필요 라이브러리 호출
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
df = pd.read_csv('data/naver_comment.csv', encoding='utf8')
df.head()

,title,score,score_num,best_num,best,best_recomm,best_unrecomm,comment_num,comment,star
0,엑스트라 데이즈,9.62,101,6,아 근데 실제로 저런 기술이 있다면 쫌 싸할것같다.. 내가 죽어도 사람들은 나랑 똑...,126,0,42,버키 왜 하나만 블러예욬ㅋㅋㅋㅋㅋ\n이 웹툰은 개재밌을거라고 나한테 텔레파시가왔다\...,9.65
1,원수를 사랑하라,9.95,31400,15,근데 여주 대단하다...엄마랑 아빠가 저런 사람들인데 멀끔히 차려입고 면접보는 거면...,129323,3789,984,그림체 마음에 드네 \n그림체 미친\n면접관중에 존나 보기싫은얼굴이보이는데?\n누가...,9.92
2,우투리,8.50,159,4,"학원액션물 만화계의 거장 임재원님의 2번째 작품 우투리입니다.... 스토리, 그림체...",194,5,51,진짜 퀄리티 말도 안되는데 덴마 이후로 미완결 작품은 그냥 혐오스럽다\n우와아아아!...,9.20
3,롭플롭,9.78,290,5,"나 이 작가 작품 좋아하네.... 잭슨의 관, 데빌샷 보고 바로 들어옴 \n잭슨의 ...",410,2,64,작가님 어케 이렇게 그천이신거에요\n노랑머리 친구 명암때문인진 몰라도장례식 이후에 ...,9.91
4,약초마을 연쇄살초사건,9.73,10226,15,아니ㅋㅋㅋㅋㅋㅋ작가님ㅋㅋㅋㅋㅋㅋ이제 작물 쪽으로 길을 트신건가요ㅋㅋㅋㅋㅋㅋㅋㅋ\n팀...,62851,254,1463,ㅋㅋㅋㅋㅋㅋㅋㅋ\n조낸 무서운 이야기인데 약초란 사실과 그림체 때문에 잔혹함이 전달...,9.87


In [3]:
okt = Okt()
def okt_tokenizer(text):
    tokens_ko = okt.morphs(text, stem=True)
    return tokens_ko

import re

def hangul(text):
    return re.sub("[^가-힣ㄱ-하-ㅣ\\s]", "", text)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def train_to_evaluate(X, y, file_name, max_words=10000, maxlen=30, embedding_dim=100, *args):
    
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    X_seq = tokenizer.texts_to_sequences(X)

    X_pad = pad_sequences(X_seq, maxlen=maxlen)
    y = np.asarray(y)
    
    X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=0)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
    
    model = Sequential()
    model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
    model.add(Flatten())
    for unit in args:
        model.add(Dense(unit, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    cp = ModelCheckpoint(filepath='data/'+file_name,
                         monitor='accuracy',
                         save_best_only=True)
    
    es = EarlyStopping(monitor='accuracy',
                       patience=15)
    
    
    hist = model.fit(X_train, y_train,
                     epochs=100, batch_size=32,
                     validation_data=(X_val, y_val),
                     callbacks=[cp, es],
                     verbose=0)
    
    
    model.load_weights('data/'+file_name)
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
#     print('MAE :', np.round(mae, 4), 'RMSE :', np.round(np.sqrt(mse), 4))
        
    return model, hist, loss, accuracy

In [5]:
# 하위 25%=9.77점이므로, 9.8점 이하면 0, 9.8점 초과면 1로 이진 분류
def category(star):
    if star > 9.8:
        return 1
    else:
        return 0

# 베댓만

## 1. Dense 층 없이

In [7]:
X1 = df['best']
y = df['star'].apply(category)

X1 = X1.apply(okt_tokenizer).values

In [14]:
from tqdm import tqdm

bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'loss', 'accuracy'])

for mxwords in tqdm(range(10, 100+1, 10)):
    for mxlen in [20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            model = train_to_evaluate(X1, y, 'embedding1_category.h5', mxwords, mxlen, embdim)
            result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, model[2], model[3]]
            if bestacc < model[3]:
                bestmodel = model
                bestacc = model[3]
                bestmxwords = mxwords
                bestmxlen = mxlen
                bestembdim = embdim
                
bestmodel[0].save('embedding1_category.h5')

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [45:25<00:00, 272.54s/it]


In [15]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
20,10.0,25.0,15.0,0.654153,0.666667
172,40.0,20.0,15.0,0.673053,0.682927
57,20.0,20.0,10.0,0.708638,0.666667
440,80.0,30.0,25.0,0.713858,0.674797
512,90.0,30.0,100.0,0.737767,0.634146
7,10.0,20.0,45.0,0.739302,0.707317
442,80.0,30.0,35.0,0.742826,0.674797
5,10.0,20.0,35.0,0.743784,0.707317
447,80.0,30.0,60.0,0.753499,0.666667
444,80.0,30.0,45.0,0.754436,0.674797


In [17]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
337,60.0,30.0,80.0,0.768579,0.715447
141,30.0,25.0,50.0,1.192750,0.707317
5,10.0,20.0,35.0,0.743784,0.707317
326,60.0,30.0,25.0,0.773254,0.707317
7,10.0,20.0,45.0,0.739302,0.707317
324,60.0,30.0,15.0,0.820030,0.707317
316,60.0,25.0,70.0,0.822522,0.707317
135,30.0,25.0,20.0,1.074013,0.707317
311,60.0,25.0,45.0,0.846052,0.707317
335,60.0,30.0,70.0,0.842144,0.699187


## 특문 제거

In [20]:
X1a = df['best'].apply(hangul)
X1a = X1a.apply(okt_tokenizer).values

In [23]:
bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'loss', 'accuracy'])

for mxwords in tqdm(range(10, 100+1, 10)):
    for mxlen in [20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            model = train_to_evaluate(X1a, y, 'embedding1a_category.h5', mxwords, mxlen, embdim)
            result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, model[2], model[3]]
            if bestacc < model[3]:
                bestmodel = model
                bestacc = model[3]
                bestmxwords = mxwords
                bestmxlen = mxlen
                bestembdim = embdim
                
bestmodel[0].save('embedding1a_category.h5')

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [49:57<00:00, 299.79s/it]


In [24]:
print(f'Accuracy: {bestmodel[3]:.4f}, loss: {bestmodel[2]:.4f}')
print(f'--bestmxwords: {bestmxwords}, --bestmxlen: {bestmxlen}, --bestembdim: {bestembdim}')

Accuracy: 0.7398, loss: 0.7582
--bestmxwords: 90, --bestmxlen: 20, --bestembdim: 40


In [25]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
20,10.0,25.0,15.0,0.637171,0.666667
565,100.0,30.0,80.0,0.670706,0.715447
285,60.0,20.0,10.0,0.672909,0.682927
456,90.0,20.0,10.0,0.675690,0.666667
563,100.0,30.0,70.0,0.680825,0.691057
510,90.0,30.0,90.0,0.684096,0.707317
507,90.0,30.0,75.0,0.685557,0.699187
569,100.0,30.0,100.0,0.688168,0.691057
503,90.0,30.0,55.0,0.690852,0.691057
501,90.0,30.0,45.0,0.691844,0.691057


In [26]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,loss,accuracy
462,90.0,20.0,40.0,0.758202,0.739837
509,90.0,30.0,85.0,0.712707,0.731707
256,50.0,25.0,55.0,0.824740,0.723577
553,100.0,30.0,20.0,0.828280,0.715447
251,50.0,25.0,30.0,0.908665,0.715447
506,90.0,30.0,70.0,0.736070,0.715447
327,60.0,30.0,30.0,0.827564,0.715447
259,50.0,25.0,70.0,0.834172,0.715447
459,90.0,20.0,25.0,0.742436,0.715447
565,100.0,30.0,80.0,0.670706,0.715447


## 2. Dense 층 추가

In [28]:
bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'unit', 'loss', 'accuracy'])

for mxwords in tqdm(range(20, 60+1, 10)):
    for mxlen in [10, 15, 20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            for unit in [16, 32, 48, 64]:
                model = train_to_evaluate(X1, y, 'embedding1b_category.h5', mxwords, mxlen, embdim, unit)
                result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, unit, model[2], model[3]]
                if bestacc < model[3]:
                    bestmodel = model
                    bestacc = model[3]
                    bestmxwords = mxwords
                    bestmxlen = mxlen
                    bestembdim = embdim
                    bestunit = unit
                
bestmodel[0].save('embedding1b_category.h5')

100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [1:56:09<00:00, 1393.92s/it]


In [29]:
print(f'Accuracy: {bestmodel[3]:.4f}, loss: {bestmodel[2]:.4f}')
print(f'--bestmxwords: {bestmxwords}, --bestmxlen: {bestmxlen}, --bestembdim: {bestembdim}, --bestunit: {bestunit}')

Accuracy: 0.7073, loss: 1.1665
--bestmxwords: 20, --bestmxlen: 20, --bestembdim: 10, --bestunit: 48


In [30]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
1758,60.0,25.0,20.0,48.0,0.836050,0.642276
1876,60.0,30.0,75.0,16.0,0.836954,0.650406
1792,60.0,25.0,65.0,16.0,0.840649,0.658537
318,20.0,30.0,25.0,48.0,0.855126,0.650406
1884,60.0,30.0,85.0,16.0,0.857306,0.666667
1763,60.0,25.0,25.0,64.0,0.858631,0.691057
1777,60.0,25.0,45.0,32.0,0.864183,0.666667
1856,60.0,30.0,50.0,16.0,0.866129,0.674797
1885,60.0,30.0,85.0,32.0,0.872586,0.682927
1851,60.0,30.0,40.0,64.0,0.872787,0.642276


In [31]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
154,20.0,20.0,10.0,48.0,1.166520,0.707317
331,20.0,30.0,40.0,64.0,1.008769,0.699187
1836,60.0,30.0,25.0,16.0,0.937900,0.699187
348,20.0,30.0,65.0,16.0,0.983687,0.699187
309,20.0,30.0,15.0,32.0,1.058420,0.699187
1082,40.0,30.0,30.0,48.0,1.192200,0.691057
322,20.0,30.0,30.0,48.0,1.000705,0.691057
336,20.0,30.0,50.0,16.0,1.095724,0.691057
358,20.0,30.0,75.0,48.0,0.998079,0.691057
653,30.0,25.0,65.0,32.0,1.065130,0.691057


## 특문 제거

In [32]:
bestmodel = None
bestacc = 0
bestmxwords = None
bestmxlen = None
bestembdim = None
bestunit = None
result_df = pd.DataFrame(columns=['maxwords', 'maxlen', 'embedding_dim', 'unit', 'loss', 'accuracy'])

for mxwords in tqdm(range(20, 60+1, 10)):
    for mxlen in [10, 15, 20, 25, 30]:
        for embdim in range(10, 100+1, 5):
            for unit in [16, 32, 48, 64]:
                model = train_to_evaluate(X1a, y, 'embedding1c_category.h5', mxwords, mxlen, embdim, unit)
                result_df.loc[len(result_df)] = [mxwords, mxlen, embdim, unit, model[2], model[3]]
                if bestacc < model[3]:
                    bestmodel = model
                    bestacc = model[3]
                    bestmxwords = mxwords
                    bestmxlen = mxlen
                    bestembdim = embdim
                    bestunit = unit
                
bestmodel[0].save('embedding1c_category.h5')

100%|████████████████████████████████████████████████████████████████████████████████| 5/5 [1:47:31<00:00, 1290.33s/it]


In [34]:
print(f'Accuracy: {bestmodel[3]:.4f}, loss: {bestmodel[2]:.4f}')
print(f'--bestmxwords: {bestmxwords}, --bestmxlen: {bestmxlen}, --bestembdim: {bestembdim}, --bestunit: {bestunit}')

Accuracy: 0.7398, loss: 1.0082
--bestmxwords: 50, --bestmxlen: 25, --bestembdim: 10, --bestunit: 32


In [36]:
result_df.sort_values(by=['loss']).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
1473,50.0,30.0,45.0,32.0,0.851340,0.634146
1868,60.0,30.0,65.0,16.0,0.881700,0.723577
1840,60.0,30.0,30.0,16.0,0.885032,0.723577
1393,50.0,25.0,40.0,32.0,0.885849,0.682927
1394,50.0,25.0,40.0,48.0,0.887846,0.731707
1404,50.0,25.0,55.0,16.0,0.907192,0.691057
1884,60.0,30.0,85.0,16.0,0.911642,0.691057
695,30.0,30.0,20.0,64.0,0.916271,0.666667
1388,50.0,25.0,35.0,16.0,0.918320,0.707317
1800,60.0,25.0,75.0,16.0,0.920301,0.650406


In [37]:
result_df.sort_values(by=['accuracy'], ascending=False).head(10)

,maxwords,maxlen,embedding_dim,unit,loss,accuracy
1369,50.0,25.0,10.0,32.0,1.008193,0.739837
1409,50.0,25.0,60.0,32.0,0.978258,0.731707
1394,50.0,25.0,40.0,48.0,0.887846,0.731707
1868,60.0,30.0,65.0,16.0,0.881700,0.723577
1335,50.0,20.0,60.0,64.0,1.015600,0.723577
1400,50.0,25.0,50.0,16.0,0.980454,0.723577
1840,60.0,30.0,30.0,16.0,0.885032,0.723577
1302,50.0,20.0,20.0,48.0,1.041961,0.715447
1370,50.0,25.0,10.0,48.0,1.028498,0.715447
1375,50.0,25.0,15.0,64.0,1.167997,0.715447


# 테스트

In [60]:
'''
Accuracy: 0.7398, loss: 1.0082
--bestmxwords: 50, --bestmxlen: 25, --bestembdim: 10, --bestunit: 32
'''
max_words = 50
maxlen = 25

from tensorflow.keras.models import load_model

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X1a)
X_seq = tokenizer.texts_to_sequences(X1a)
X_pad = pad_sequences(X_seq, maxlen=maxlen)
y = np.asarray(y)

X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [61]:
model = load_model('embedding1c_category.h5')

In [63]:
model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 2ms/step - loss: 1.1504 - accuracy: 0.7013


[1.15042245388031, 0.701298713684082]

In [72]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(df['best'], df['star'], test_size=0.2, random_state=0)

In [74]:
pred = model.predict(X_test)

In [84]:
y_test_orig[y_test_orig < 9.8]

14     9.66
338    9.69
395    9.51
204    9.54
546    9.56
175    9.73
172    9.73
79     9.75
374    9.75
40     9.67
181    9.77
8      9.73
271    8.64
215    9.78
756    9.78
452    7.14
303    9.69
451    7.95
272    9.63
250    9.21
253    9.63
672    9.28
499    9.38
85     9.49
526    9.40
735    9.79
242    4.96
18     9.50
623    9.78
251    9.71
50     9.60
443    9.79
2      9.20
760    9.75
586    8.01
187    7.07
283    9.72
279    9.65
258    9.39
37     7.48
48     6.28
64     9.77
230    9.38
Name: star, dtype: float64

In [87]:
y_test_orig_copy = y_test_orig.reset_index(drop=True)

In [88]:
y_test_orig_copy[y_test_orig_copy < 9.8]

3      9.66
6      9.69
8      9.51
9      9.54
11     9.56
16     9.73
22     9.73
28     9.75
33     9.75
39     9.67
40     9.77
44     9.73
46     8.64
52     9.78
53     9.78
54     7.14
57     9.69
58     7.95
67     9.63
68     9.21
72     9.63
76     9.28
77     9.38
82     9.49
83     9.40
84     9.79
86     4.96
87     9.50
89     9.78
91     9.71
93     9.60
105    9.79
109    9.20
110    9.75
112    8.01
113    7.07
115    9.72
116    9.65
117    9.39
135    7.48
137    6.28
144    9.77
151    9.38
Name: star, dtype: float64

In [89]:
X_test_orig.iloc[86]

'주변 사람들 반응 묘사하며... 여주가 스토킹행위에 대해 점점 아무렇지 않게 생각하는 등 여러모로 범죄미화의 요소가 많습니다. 남자주인공은 몰카를 찍고 스토킹을 하는 범죄자에요...\n미워할 수 없는 훈남스토커? 제정신인가\n오래 된 작품을 독자들이 왜 감안하고 봐야하는지 이해를 못 하겠네요. 애초에 범죄를 미화 시키는 내용을 담은 작품이 잘못 된 것이지.. 웹툰 플랫폼에는 중학생 고등학생 성인 뿐만 아니라 아직 스토킹에 대해 잘 모르는 어린 초등학교 저학년도 있어요. 그런데 그 아이들이 이 작품을 본다? 그러면 어떻게 되는지는 말 안 해도 아시잖아요. 그것이 아이의 뇌에 인식이 되어서 결국은 범죄율이 높아지죠. 베스트 도전에서 가지고 오지 않는 것도 이해가 가지않지만, 타 플랫폼에서 왜 이런 작품을 걸러내지 않고 가져오는지, 그리고 이 작품을 왜 옹호하는지도 이해가 가지않네요.\n이게 뭔 내용이냐 ㅅㅂㅋㅋㅋㅋㅋ 스토킹 이런 걸 떠나서 너무 허접함 \n네이버 깡 좋네 지금 세대에 이런 만화를 올리다니ㅋㅋ\n이걸 미화시키네\n남주 소름돋는다 진짜...\n제목부터 으 하면서 들어왔습니다. 스토킹은 엄연한 범죄고 미화되서는 안 된다고 생각합니다 ㅠㅠ. \n저오래된 작품인걸 감안해야 하는 이유가있나싶어요. 오래된거니까 우리가 이해하고 참고봐야된다? 이런건 아니잖아요... 시대가 변하면서 인식도 변했기 때문에 이런 작품을 보는 시선또한 변하는게거구요. 우리가 이작품이 나왓던 시대의 생각에 맞출필요는없다고 생각해요.\n네이버웹툰 완결작이나 데려오지 스토킹을해도 훈남이라 미워할수 없다는둥의 작품을 가져오네 타 플랫폼껄 데려와도 좀 정상적이고 스토리 탄탄한거 데려올 순 없나? 어린이도 보는 웹툰에 이런 작품은 너무 별로다 -_-;;\n웹툰 첫 컷에 범죄라고 크게 적으면 뭐해 그게 정당화가 되나 이미 미워할 수 없는 스토커라는 작가에 말부터가 글렀는데 이건 시대를 떠나서 원래 큰 범죄임. 그리고 여자는 남자 남자는 여자 없으면 못 사는 것처럼 짐승마냥 표현하는 것도 그렇다 

In [90]:
y_test_orig.iloc[86], pred[86]

(4.96, array([0.6954117], dtype=float32))

In [91]:
X_test_orig.iloc[137]

'밑그림 남아있는건 좀 너무하지 않냐 한 컷도 아니고 대부분의 컷이 그럼\n네웹 어디 학원에서 넣어준다는 소문 있던데 진짜인가 보네...\n털선을 자기 아이덴티티로 밀고 갈 생각인진 모르겠지만 이건 그림을 잘 그리고 못 그리기 이전에 완성도와 성의의 문제임. 연습장에 그리다 만 그림같은 퀄리티를 네이버에서 정식연재한다고?\n선 정리 조금만 해주면 진짜 좋을것 같은데 그점 빼고는 다 좋은거 같네요\n내용이랑 그림체는 많이안나쁜데..잡선이 너무많이보이네요 파일잘못 올리셨낭..?\n작가님 한텐 좀 죄송하긴 하다만 솔직히 네이버 요즘 일 안하는거같음 베도만 가 봐도 그림이쁘고 깔끔하고 스토리 탄탄한거 좀만 둘러봐도 나오는데。。。 이건 좀 아닌것같음\n작가가털선쓰는건첨보네\n이런 그림은 개성이 아니라 못그린거야 개성은 다른거지\n진짜 죄송한데..이건 뭐 느낌이 있는 그림도 아닌거 같구 털선 너무 보기가 싫음 털선임..가끔 털선이여도 예쁜 털선이 있는데 이건 걍 털선 그 자체.. 털선때문에 내용에 집중을 못하겠음..\n뭐야.. 베도에 이거보다 퀄 좋고 스토리 구성 좋은 작품 많은데.. 왜 베도도 아니고 이런 퀄을 그리는 작가들을 데리고 오는거지?\n털선 이어붙이기가 좀 심함 그림 그리는 사람이라면 털선은 모두가 싫어하는데\n그림을 학원에서 배우던 독학을 하던 먼저 해야하는건 선 연습이에요.그 기초도 안되어있으니 사람들의 시선이 불편하죠..\n밑밑밑댓 미술 안 배워본듯. 선 좀 지저분한 걸 떠나서 구도 인체비율 투시 틀린 거 많음 ㅋㅋ 당장 배경만 해도 투시 다 틀렸는데? 앵글에 맞지 않게 사람 넣은 컷도 보이고. 인체비율은 작가 개성이라고 봐도 문제될 거 없지만 배경 투시는 진짜 수학책 입체도형 자료 급으로 위아래 선이 평행한데 뭔 투시가 맞아 ㅋㅋ 소실점이 어딨는지도 못찾겠더라 난.. 그리고 물타기물타기 하는데 오히려 지금 작화 감싸는 게 더 물타기같다고 생각됨. 개성있고 보기 좋다고? 아니 못그리는 걸 못그린다고 말하는게 어떻게 물타기야 ㅋㅋㅋ 보기 싫으면 보지 말라

In [92]:
y_test_orig.iloc[137], pred[137]

(6.28, array([0.99188083], dtype=float32))

In [93]:
X_test_orig.iloc[113]

'베댓 말대로 그냥 쟤 저렇게 살았으면 좋겠다. 중학교 내내 나 괴롭히던 애도 고등학생 때 정신 차려서 대학도 가고 이번에 취업도 했던데 나는 너 때문에 정신병원에 일년 넘게 갇혀 있었고 집 밖에 나오는 데까지 꼬박 삼 년 걸렸다. 이러는 데도 너는 sns에 취업 축하한다는 댓글 밑에 대단한 회사도 아니다 게시글에는 그저그런 인생 이런 말 하던데 나는 너 때문에 그저그런 인생도 못 살고 아직도 트라우마에 빠져 살고 있어. 너한테는 내가 과거의 실수고 죄책감들 중에 하나겠지만 나한테 너는 너무 큰 벽이다. 아직도.\nㅋㅋㅋㅋㅋㅋㅋ 머리도 염색하고 패싸움에 전학오자마자 한명 날려버리고 유리창 파손까지 그래놓고 조용히살고싶다 ㅋㅋㅋㅋㅋ\n나는 쟤 그냥 계속 저렇게 살면 좋겠는데?일진이여서 폭력 행사하다가 이제서야 대학간다고 정신차린척 해봤자 자기가 일진일때 왕따를 시키거나 때리거나 한 일은 없어지지 않잖아...피해자들은 얼마나 슬프고 증오스럽겠어 일진미화 그만 좀 나오면 좋겠다. 소재가 이런거 밖에 없나..애들 때리는 과거 회상도 나오고 쟤가 예전에 불량하게 살았다는게 적혀있는데 도대체 뭘 오바했다는 건지.. 쟤가 과거청산 하고 대학가려는게 역겹다고...  그런 내용이니깐 일진 미화지.팩트를 말하면 그냥 듣지 뭐가 아니꼬와서 싫어요나 누르고 있음..ㅋㅋㅋㅋ\n도망가다 가해자놈 차에치여죽는엔당으로하고 2화부터 다른주인공 나와도 모르는척할게요\n찐찐거릴게 아니라 저런걸 멋있다 생각하고 동경하는 니네가 찐인듯ㅠㅠ❗ 아닌걸 아니라 말하는데 거따대고 툭하면 만물찐따설 웅앵웅앵 지겹지도 않낭~~!\n아직은 일진미화인지 모르는 거 맞는데 주인공이나 일진들 옹호하지 말라구... 법이랑 질서 다 어기는 게 좋은 건 아니잖아\n이게 일진 미화지 뭘 아니야... ㅋㅋㅋ 가오 부리는 댓글들 진짜 혐오스럽다\n일진만화 베댓 특:첫화엔 욕하다가 재밌다면서 계속봄\n통이라고했지 일진놀이하는애로는 안보이는데 쟤가 자기보다 약한애들괴롭혀서 통이됐을지 자기건드리는애들만 족쳐서 통이됐을지 누가안다고

In [94]:
y_test_orig.iloc[113], pred[113]

(7.07, array([0.89948404], dtype=float32))

결과는 별로다.  
데이터 수집 시점에서 타 플랫폼에서 넘어온 작품이 완결로 넘어와서 베댓 수가 적으면서 점수가 낮은 작품 때문에 정확도가 낮은 것 같다.  
또는 3:1 정도로 데이터가 치중되서 그런 것일 수도 있다. (아니면 애초에 댓글로 점수를 예측한다는 가정부터 틀린 것일지도)  
우선 데이터 재수집(작품의 하트(관심도)가 100개 이상인 작품만 수집) + 1:1:1:1로 4등분하여 재시도 하는 걸로